In [ ]:
import requests
import os
import scanpy as sc
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import gc


In [ ]:
import scanpy as sc
print(sc.__version__)

## Set parameters and paths

In [ ]:
# Define parameters
res = 1
n_hvg = 2000

In [ ]:
# Define the input and output directories
input_dir = 'Input_files'
output_dir = 'Output_Gabritto_2024'

# Create the output directory if it does not exist
if not os.path.exists(input_dir):
    os.makedirs(input_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Analysis for Gabritto_2024

### Load data

In [ ]:
studyID = 'Gabritto_2024'
download_link = 'https://datasets.cellxgene.cziscience.com/145aadbe-fd91-4592-bf14-3b4c644a2141.h5ad'
new_filename = f"Input_files/{studyID}.h5ad"
response = requests.get(download_link)
with open(new_filename, 'wb') as f:
        f.write(response.content)
print(f"File downloaded and saved as {new_filename}")

### Preprocess and clustering

In [ ]:
file = 'Gabritto_2024.h5ad'
print(f'Start process {studyID}')
adata=sc.read_h5ad(os.path.join(input_dir,file))
print(f'    Before process: number of obs {adata.n_obs}, number of var {adata.n_vars}')

adata.layers["counts"]=adata.X.copy()
adata.var['GeneID']=adata.var.index
adata.var.set_index('feature_name',inplace=True)
if adata.raw is not None:
    adata.raw.var.set_index('feature_name', inplace=True)
else:
    print("    adata.raw is None, skipping setting index.")
adata.var["mt"]=adata.var_names.str.startswith("MT-")

adata.obs['nCount_RNA'] = adata.obs['Number of UMIs']
adata.obs['nFeature_RNA'] = adata.obs['Genes detected']
del adata.obs['Number of UMIs']
del adata.obs['Genes detected']

# QC
print(f'    Start QC')
sc.pp.calculate_qc_metrics(adata,qc_vars=["mt"],inplace=True,log1p=True)
sc.pp.filter_genes(adata,min_cells=3)
adata=adata[adata.obs['nFeature_RNA']>200,:]
adata=adata[adata.obs['pct_counts_mt']<10,:]
adata=adata[adata.obs['nCount_RNA']>400,:]
adata=adata[adata.obs['nCount_RNA']<60000,:]
print(f'    After process: number of obs {adata.n_obs}, number of var {adata.n_vars}')

print(f'    It is Normalized and Log-Transformed data, no need to run normalization')

sc.pp.highly_variable_genes(adata,n_top_genes=n_hvg)
sc.tl.pca(adata)
explained_variance_ratio = adata.uns['pca']['variance_ratio']
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Elbow Plot for PCA')
plt.grid(True)
plt.show()
sc.pp.neighbors(adata,n_neighbors=10,n_pcs=30)
sc.tl.umap(adata)
sc.tl.leiden(adata,resolution=res,n_iterations=2)

### Remove non-microglials

In [ ]:
goi=['PTPRC', 'LYVE1', 'MRC1', 'CD163','PTPRC', 'CD3E', 'CD79B', 'SKAP1', 'ETS1','PTPRC', 'FCN1', 'VCAN', 'FGR', 'CD300E','SATB1']
goi_in_adata=[gene for gene in goi if gene in adata.var_names]
sc.pl.dotplot(adata, goi_in_adata, groupby="leiden");

In [ ]:
# remove clusters
clusters_to_remove = ['13', '15', '18']
adata = adata[~adata.obs['leiden'].isin(clusters_to_remove), :]
adata

In [ ]:
# Reclustering
import scanpy as sc
import matplotlib.pyplot as plt
res = 0.4
sc.pp.highly_variable_genes(adata,n_top_genes=n_hvg)
sc.tl.pca(adata)
sc.pp.neighbors(adata,n_neighbors=10,n_pcs=30)
sc.tl.umap(adata)
sc.tl.leiden(adata,resolution=res,n_iterations=2)

### Figure 7F

In [ ]:
sc.pl.umap(adata, color=['leiden'], palette='Paired',legend_loc='on data', save=f'_{studyID}_microglia_leiden_res{res}.pdf')    

In [ ]:
goi=['P2RY12','TMEM119','AIF1','C1QB','CX3R1','GPR34','CSF1R','SLCO2B1','TBXAS1','DOCK8','APBB1IP','PCDH9']

# Create a new variable 'cluster2' in adata.obs
adata.obs['cluster2'] = adata.obs['leiden'].apply(lambda x: '5' if x == '5' else 'Other_clusters')
adata.obs['cluster2'] = adata.obs['cluster2'].astype('category')

goi_in_adata=[gene for gene in goi if gene in adata.var_names]
sc.pl.dotplot(adata, goi_in_adata, groupby="cluster2",save=f'{studyID}_microglia_goi_res{res}_check_microgliaMarker.pdf')

In [ ]:
# Importing necessary libraries
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

# Define the markers you are interested in
markers = ["CD22",'BHLHE41']

# Loop over each marker to create a density plot on UMAP
for marker in markers:
    # Create a UMAP scatter plot colored by marker expression
    sc.pl.umap(adata, color=marker, show=False, cmap='viridis', legend_loc=None,s=20)
    
    # Overlay density (this step is similar to Nebulosa, but done manually)
    ax = plt.gca()  # Get current axis
    
    # Extract UMAP coordinates
    x = adata.obsm['X_umap'][:, 0]  # UMAP 1 coordinates
    y = adata.obsm['X_umap'][:, 1]  # UMAP 2 coordinates
    
    # Extract expression levels of the marker
    expression = adata[:, marker].X.toarray().flatten()
    
   # Create a 2D histogram / density plot
    hexbin_plot = ax.hexbin(x, y, C=expression, gridsize=70, cmap='inferno', reduce_C_function=np.mean, mincnt=1)

    # Set axis labels and title
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    plt.title(f'Density plot for {marker}')

    # Add the colorbar with the mappable hexbin plot
    plt.colorbar(hexbin_plot, label='Density')
    
    # Save or show the figure
    plt.savefig(f'figures/umap_density_{studyID}_{marker}_within_macrolia.pdf')
    plt.show()

#### BHLHE41/CD22 in different clusters

In [ ]:
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import fisher_exact
from scipy.stats import ttest_ind

# Select cells from Leiden cluster 5 and all other clusters
cluster_5_cells = adata[adata.obs['leiden'] == '5']
other_clusters_cells = adata[adata.obs['leiden'] != '5']

# Extract expression levels of BHLHE41 and CD22 for cluster 5 and other clusters
bhlhe41_expression_cluster_5 = cluster_5_cells[:, 'BHLHE41'].X.toarray().flatten()
cd22_expression_cluster_5 = cluster_5_cells[:, 'CD22'].X.toarray().flatten()

bhlhe41_expression_other_clusters = other_clusters_cells[:, 'BHLHE41'].X.toarray().flatten()
cd22_expression_other_clusters = other_clusters_cells[:, 'CD22'].X.toarray().flatten()

# Add 0.01 to the expression levels
bhlhe41_expression_cluster_5 += 0.01
cd22_expression_cluster_5 += 0.01

bhlhe41_expression_other_clusters += 0.01
cd22_expression_other_clusters += 0.01

# Calculate the value of BHLHE41/CD22
bhlhe41_cd22_ratio_cluster_5 = bhlhe41_expression_cluster_5 / cd22_expression_cluster_5
bhlhe41_cd22_ratio_other_clusters = bhlhe41_expression_other_clusters / cd22_expression_other_clusters

# Perform log transformation
bhlhe41_cd22_ratio_cluster_5_log = np.log1p(bhlhe41_cd22_ratio_cluster_5)
bhlhe41_cd22_ratio_other_clusters_log = np.log1p(bhlhe41_cd22_ratio_other_clusters)

# Create a DataFrame for plotting
data = {
    'BHLHE41_CD22_Ratio': np.concatenate([bhlhe41_cd22_ratio_cluster_5_log, bhlhe41_cd22_ratio_other_clusters_log]),
    'Group': ['Cluster 5'] * len(bhlhe41_cd22_ratio_cluster_5_log) + ['Other Clusters'] * len(bhlhe41_cd22_ratio_other_clusters_log)
}

df = pd.DataFrame(data)

# Calculate the median value for each group
median_values = df.groupby('Group')['BHLHE41_CD22_Ratio'].median()
print('Median value:',  median_values)

# Plot the data
fig, ax = plt.subplots()
df.boxplot(column='BHLHE41_CD22_Ratio', by='Group', ax=ax)

# Perform t-test to get the p-value between the two groups
t_stat, p_value = ttest_ind(bhlhe41_cd22_ratio_cluster_5_log, bhlhe41_cd22_ratio_other_clusters_log)
print(p_value)

plt.title('Boxplot of BHLHE41_CD22_Ratio by Group')
plt.suptitle('')
plt.xlabel('Group')
plt.ylabel('BHLHE41_CD22_Ratio')
plt.savefig(f'figures/{studyID}_microglia_Cluster5vsOthers.pdf')
plt.show()
